**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-89oc_p8t
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-89oc_p8t
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-89oc_p8t did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-89oc_p8t did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,fe060143e239dfa55af69982d0dbc619e2228b93,93ccda8a38077612a8f32b8bf1b430f8439cf114fefb22...,2015-09-21 15:15:00+00:00,2015-09-21 15:30:00+00:00,360,0.70,1.703184e+10,1.703128e+10,32,28,...,0.0,5.45,Cash,None,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.885300,-87.642808,POINT (-87.6428084655 41.8853000224)
1,9e6a37a0cdfe479eebe5de0fa250fda7792448d3,2120ffd6768c3ddb1d831afa3f2ac12eada1fbde60bf4d...,2015-10-01 18:30:00+00:00,2015-10-01 18:45:00+00:00,780,1.53,1.703108e+10,1.703124e+10,8,24,...,1.0,10.05,Cash,None,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.892658,-87.652534,POINT (-87.6525344838 41.8926581076)
2,422f9087772c73d3148f4155204f554daebac100,d5dc4a704805faed89d18144387c3f5f9150df85c6ea5d...,2015-09-20 18:30:00+00:00,2015-09-20 19:00:00+00:00,1320,2.80,NaN,NaN,4,14,...,0.0,12.45,Cash,None,41.975171,-87.687516,POINT (-87.68751551520002 41.9751709433),41.968069,-87.721559,POINT (-87.7215590627 41.968069)
3,72297eeaf07f7882d38447fe19f91b3901501c93,43946e4da8905d64ceeed7cfc2b1969df72ecf991b865a...,2015-09-21 20:45:00+00:00,2015-09-21 21:15:00+00:00,1140,17.10,1.703198e+10,1.703128e+10,76,28,...,3.0,45.65,Credit Card,None,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.879255,-87.642649,POINT (-87.642648998 41.8792550844)
4,f4d252be1d83b74abd02216ea67a80e71a16a8a4,9887bb2fa063dfbc2cdb90e985162116e86cdad1ca041e...,2015-09-23 23:00:00+00:00,2015-09-23 23:15:00+00:00,720,3.80,1.703106e+10,1.703108e+10,6,8,...,0.0,15.25,Credit Card,None,41.942585,-87.656644,POINT (-87.6566440918 41.9425851797),41.907492,-87.635760,POINT (-87.6357600901 41.9074919303)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.



In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING                 
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-03-01,91980.000000
1,2016-01-14,86916.428571
2,2016-01-02,72801.000000
3,2016-02-02,79947.285714
4,2016-03-30,74274.400000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.


In [4]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (PARTITION BY pickup_community_area ORDER BY trip_start_timestamp) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,4.0,2013-10-03 00:00:00+00:00,2013-10-03 00:15:00+00:00,1
1,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
2,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:30:00+00:00,2
3,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2
4,4.0,2013-10-03 00:15:00+00:00,2013-10-03 00:15:00+00:00,2


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [5]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,06895917531066707e5e5f4dc3564e3ef6c18853f31d27...,2013-10-03 13:00:00+00:00,2013-10-03 13:30:00+00:00,690.0
1,0e8726922a5536f2c07eb081776484ac6ed4be5b2f5b09...,2013-10-03 18:45:00+00:00,2013-10-03 18:45:00+00:00,315.0
2,22b6b96b6772f59c3cf20e1454d31a32462eec5593da20...,2013-10-03 23:00:00+00:00,2013-10-03 23:00:00+00:00,210.0
3,38f6145c9a2b848dc1baa16fd91087e606b12bcb8757a9...,2013-10-03 15:30:00+00:00,2013-10-03 15:30:00+00:00,615.0
4,3b5ccf9479decf48844e1beab2b5dba14e75c17972c396...,2013-10-03 14:00:00+00:00,2013-10-03 14:00:00+00:00,840.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

---




*Training Session - Poludniowy Albanczyk*